# Cloning torchserve to add in the docker

In [1]:
# The dockerfile places the serve folder into the docker image
!git clone https://github.com/pytorch/serve.git

Cloning into 'serve'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 8934 (delta 53), reused 77 (delta 19), pack-reused 8785
Receiving objects: 100% (8934/8934), 35.53 MiB | 60.43 MiB/s, done.
Resolving deltas: 100% (4799/4799), done.


# Creating a docker image
### This creates a docker image from the dockerfile present in the current directory and pushes it to the ecr

In [1]:
import boto3, time, json
sess    = boto3.Session()
sm      = sess.client('sagemaker')
region  = sess.region_name
account = boto3.client('sts').get_caller_identity().get('Account')

In [2]:
import sagemaker
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)

In [4]:
bucket_name = sagemaker_session.default_bucket()
print(bucket_name)

sagemaker-us-west-1-096166425824


### This creates a repository in the ecr for us to push the docker image.

In [6]:
registry_name = 'torchserve'
!aws ecr create-repository --repository-name {registry_name}


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'torchserve' already exists in the registry with id '096166425824'


### Builds the image and pushes into the ecr

In [8]:
image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

!sudo docker build -t {registry_name}:{image_label} .
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}